### Memory 기능과 ToolNode 통합
**ToolNode와 Memory 기능**은 LangGraph에서 **이전 대화를 기억하면서 도구를 사용할 수 있는 챗봇**을 만들 수 있게 해줍니다.

- `In-Memory Checkpointer`:  
    대화 기록을 메모리에 저장하여 세션 동안 이전 대화 내용을 유지합니다.

- `RunnableConfig`:  
    각 대화 세션을 구분하기 위한 thread_id를 관리하고, 동일한 스레드에서 대화 연속성을 보장합니다.

- `도구 실행과 메모리 연동`: 
    도구 실행 결과도 대화 기록에 포함되어 다음 대화에서 참조할 수 있습니다.

#### 메모리 기능의 핵심 구성요소
1. `MemorySaver (In-Memory Checkpointer)`  
    - 대화 상태를 메모리에 저장하고 관리
    - thread_id별로 대화 기록을 분리 저장
2. `RunnableConfig`  
    - 실행 시 설정 정보 전달
    - configurable 옵션으로 thread_id 설정
3. `State 관리`
    - messages 리스트에 모든 대화 기록 누적
    - 도구 실행 결과도 ToolMessage로 저장



In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")


OPENAI_API_KEY가 설정되어 있나요?: sk-proj-WS...


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chat_models import init_chat_model


# Tavily 검색 도구 생성
tavily_tool = TavilySearchResults(
    max_results=3,  # 최대 검색 결과 수
    search_depth="advanced",  # 검색 깊이 설정
    include_answer=True,  # 직접적인 답변 포함
    include_raw_content=False,  # 원본 콘텐츠 제외 (크기 줄이기)
    include_images=False  # 이미지 제외
)

# 도구 목록 생성
tools = [tavily_tool]

# LLM 설정 및 도구바인딩
llm = init_chat_model("gpt-4o-mini", model_provider="openai")
llm_with_tools = llm.bind_tools(tools)


**상태(State) 정의 및 노드 함수 작성**

In [4]:
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

class State(TypedDict):
    # 모든 대화 메시지들 (HumanMessage, AIMessage, ToolMessage 등)
    # add_messages로 메시지를 누적하여 대화 기록 유지
    messages: Annotated[list[AnyMessage], add_messages]

# ChatBot Node 함수 정의
def chatbot_node(state: State):

    # 1. 현재 상태에서 모든 메시지 가져오기
    messages = state["messages"]
    
    # 2. LLM 호출하여 응답 생성 (도구 바인딩된 모델 사용)
    response = llm_with_tools.invoke(messages)
    
    # 3. 생성된 응답을 새로운 상태로 반환
    return {"messages": [response]}

# ToolNode 초기화
tool_node = ToolNode(tools=tools)


**메모리 기능 추가: In-Memory Checkpointer**

In [5]:
# Memory 기능을 위한 import
from langgraph.checkpoint.memory import MemorySaver

# In-Memory Checkpointer 생성
# 메모리에 대화 상태를 저장하여 세션 동안 대화 기록 유지
memory = MemorySaver()

**그래프 구성 및 메모리 연동**

In [6]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition

# 3. 상태 그래프 생성
graph_builder = StateGraph(State)

# 4. 노드 추가
graph_builder.add_node("chatbot_node", chatbot_node)
graph_builder.add_node("tools", tool_node)

# 5. 엣지 추가
graph_builder.add_edge(START, "chatbot_node")
graph_builder.add_edge("tools", "chatbot_node")

# 조건부 엣지 추가 (tool_calls가 있으면 tools로, 없으면 END로)
graph_builder.add_conditional_edges(
    "chatbot_node",
    tools_condition  # chatbot_node에서 "tools"로 갈지, "END"로 갈지 결정
)

# 중요: 메모리(checkpointer)와 함께 그래프 컴파일
# 이렇게 해야 대화 기록이 저장되고 이전 대화를 기억할 수 있습니다
graph = graph_builder.compile(checkpointer=memory)

**RunnableConfig를 사용한 Thread ID 관리**

메모리 기능을 사용하려면 **RunnableConfig**를 통해 **thread_id**를 설정해야 합니다.
- 같은 thread_id를 사용하면 이전 대화를 기억합니다
- 다른 thread_id를 사용하면 새로운 대화 세션이 시작됩니다


**메모리 기능 테스트: 이전 대화를 기억하는 챗봇**

첫 번째 대화에서 기억을 하는지 확인하고 thread_id를 변경해서 질문해보기


In [7]:
from langchain_core.runnables import RunnableConfig

user1_config = RunnableConfig(
    recursion_limit=10,
    configurable={"thread_id": "user_session_1"}
)

In [8]:
user1_input = "안녕 나는 bear 강사야"

for event in graph.stream({"messages": [("user", user1_input)]}, config=user1_config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕하세요, Bear 강사님! 어떤 도움이 필요하신가요?


In [11]:
user1_input = "내가 누구라고했지?"

for event in graph.stream({"messages": [("user", user1_input)]}, config=user1_config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

당신은 "bear 강사"라고 하셨습니다. 추가로 궁금하신 점이나 도움이 필요하신 부분이 있으면 말씀해 주세요!


In [12]:
user2_config = RunnableConfig(
    recursion_limit=10,
    configurable={"thread_id": "user_session_2"}
)

In [13]:
user2_input = "내가누구라고?"

for event in graph.stream({"messages": [("user", user2_input)]}, config=user2_config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

당신은 자신을 정의할 수 있는 독특한 개인입니다. 어떤 특정한 질문이나 주제에 대해 궁금한 점이 있다면 말씀해 주세요!


**저장된 대화 기록 확인**  
- CheckPoint 에는 현재 상태 값, 해당 구성, 처리할 next 노드가 포함되어있어 확인가능하다.

In [14]:
history = graph.get_state(user1_config)

history.values["messages"]

[HumanMessage(content='안녕 나는 bear 강사야', additional_kwargs={}, response_metadata={}, id='d9f8d1b3-5494-4dda-ba08-e1a2ccf4fa6f'),
 AIMessage(content='안녕하세요, Bear 강사님! 어떤 도움이 필요하신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 86, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CH7XH6uOs5Hf6YppBmRmamsArJ2zD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--64b546c2-fd41-48a2-9c35-72ce08733571-0', usage_metadata={'input_tokens': 86, 'output_tokens': 17, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='내가 누구라고했지?', a

In [15]:
for state in graph.get_state_history(user1_config):
    # 메시지 수 및 다음 상태 출력
    print("메시지 수: ", len(state.values["messages"]), "다음 노드: ", state.next)
    print("-" * 80)

메시지 수:  7 다음 노드:  ()
--------------------------------------------------------------------------------
메시지 수:  6 다음 노드:  ('chatbot_node',)
--------------------------------------------------------------------------------
메시지 수:  5 다음 노드:  ('__start__',)
--------------------------------------------------------------------------------
메시지 수:  5 다음 노드:  ()
--------------------------------------------------------------------------------
메시지 수:  4 다음 노드:  ('chatbot_node',)
--------------------------------------------------------------------------------
메시지 수:  3 다음 노드:  ('__start__',)
--------------------------------------------------------------------------------
메시지 수:  3 다음 노드:  ('chatbot_node',)
--------------------------------------------------------------------------------
메시지 수:  2 다음 노드:  ('__start__',)
--------------------------------------------------------------------------------
메시지 수:  2 다음 노드:  ()
--------------------------------------------------------------------------------
메